In [1]:
from faker import Faker
import random
from datetime import date, timedelta
import mysql.connector
import re

# Połączenie z bazą danych
con = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

mycursor = con.cursor()

# Inicjalizacja biblioteki Faker
fake = Faker("pl_PL")  # Generowanie danych w języku polskim

# Funkcja do usuwania polskich znaków
def remove_polish_chars(text):
    polish_chars = {
        'ą': 'a', 'ć': 'c', 'ę': 'e', 'ł': 'l', 'ń': 'n', 'ó': 'o', 'ś': 's', 'ź': 'z', 'ż': 'z',
        'Ą': 'A', 'Ć': 'C', 'Ę': 'E', 'Ł': 'L', 'Ń': 'N', 'Ó': 'O', 'Ś': 'S', 'Ź': 'Z', 'Ż': 'Z'
    }
    return ''.join(polish_chars.get(c, c) for c in text)

# Lista popularnych domen e-mail
domeny = ["gmail.com", "yahoo.com", "outlook.com", "wp.pl", "interia.pl", "onet.pl"]

# Lista stanowisk w ustalonej kolejności
stanowiska = [
    "Manager",
    "Programista",
    "Księgowy",
    "Specjalista ds. marketingu",
    "Konsultant ds. obsługi klienta",
    "Konsultant ds. obsługi klienta",
    "Konsultant ds. obsługi klienta"
]

# Funkcja do generowania pensji w zależności od stanowiska
def generuj_pensje(stanowisko):
    if stanowisko == "Manager":
        return random.randint(12000, 15000)
    elif stanowisko in ["Programista", "Księgowy"]:
        return random.randint(10000, 12000)
    elif stanowisko == "Specjalista ds. marketingu":
        return random.randint(8000, 10000)
    else:  # Dla pozostałych stanowisk
        return random.randint(5000, 8000)

# Funkcja do generowania e-maila na podstawie imienia i nazwiska, usuwanie polskich znaków
def generuj_email(imie, nazwisko):
    domena = random.choice(domeny)  # Wybieramy losową domenę z listy
    return f"{remove_polish_chars(imie.lower())}.{remove_polish_chars(nazwisko.lower())}@{domena}"

# Funkcja do generowania daty urodzenia na podstawie stanowiska
def generuj_date_urodzenia(stanowisko):
    if stanowisko == "Manager":
        return fake.date_of_birth(minimum_age=30, maximum_age=45)  # Manager musi mieć przynajmniej 30 lat
    elif stanowisko in ["Programista", "Księgowy", "Specjalista ds. marketingu"]:
        return fake.date_of_birth(minimum_age=25, maximum_age=45)  # Programista, Księgowy, Marketing mają przynajmniej 25 lat
    else:  # Dla pozostałych stanowisk
        return fake.date_of_birth(minimum_age=20, maximum_age=45)  # Pozostali mają przynajmniej 20 lat

# Funkcja do losowania daty zatrudnienia z różnymi przedziałami
def losuj_date_zatrudnienia(stanowisko):
    if stanowisko in ["Manager", "Programista"]:
        start_date = date(2022, 1, 1)
        end_date = date(2022, 1, 5)  # Zakres dla Managera i Programisty
    else:
        start_date = date(2022, 1, 5)
        end_date = date(2022, 1, 14)  # Zakres dla pozostałych stanowisk
    
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

# Generowanie i wstawianie danych do tabeli
for stanowisko in stanowiska:
    imie = fake.first_name()
    nazwisko = fake.last_name()

    # Generowanie e-maila na podstawie imienia i nazwiska, usuwanie polskich znaków
    email = generuj_email(imie, nazwisko)

    # Generowanie numeru telefonu z prefiksem +48
    numer_telefonu = "+48 " + " ".join([str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)])

    adres = fake.address().replace("\n", ", ")  # Zamiana nowej linii na przecinek
    pensja = generuj_pensje(stanowisko)  # Generowanie pensji w zależności od stanowiska
    data_urodzenia = generuj_date_urodzenia(stanowisko)  # Generowanie daty urodzenia w zależności od stanowiska
    data_zatrudnienia = losuj_date_zatrudnienia(stanowisko)

    # Wstaw dane do tabeli
    mycursor.execute(""" 
        INSERT INTO Pracownicy (Imie, Nazwisko, Stanowisko, Numer_telefonu, Email, Adres, Pensja, Data_urodzenia, Data_zatrudnienia)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (imie, nazwisko, stanowisko, numer_telefonu, email, adres, pensja, data_urodzenia, data_zatrudnienia))

# Zatwierdź zmiany i zamknij połączenie
con.commit()
mycursor.close()
con.close()
